#### @prompt example

In [ ]:
from openbb import obb
import os
obb.account.login(pat=os.environ['PAT_KEY'])

In [ ]:
obb.news.world(provider='fmp').to_df()

In [3]:
from magentic import prompt
from openbb import obb


@prompt("Explain like I'm five this financial concept: {concept}")
def explain(concept: str) -> str: ...


explain("Subprime mortgage crisis")

"Alright! Imagine you have a friend who really wants a big cookie, but they don't have enough money to buy it. So they borrow money from another friend. But here's the thing: your first friend isn't very good at saving or paying back money, kind of like when you borrow a toy and forget to give it back.\n\nNow, if lots of people who weren't very good at paying back money borrowed to buy cookies, and they couldn't pay the money back, it becomes a big problem. The friends who lent the money might get worried and not have enough to share, and soon there's a big mess because the cookies (or in real life, houses) are not being paid for.\n\nThe subprime mortgage crisis was like this, but with houses and banks instead of cookies and friends. Lots of people borrowed money to buy houses, but then they couldn't pay it back, which caused a lot of trouble for the banks and the whole money system, leading to a big, worldwide financial mess!"

In [ ]:
from magentic import prompt
from pydantic import BaseModel


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create a strong portfolio of {size} allocation size.")
def create_portfolio(size: str) -> Portfolio: ...


portfolio = create_portfolio("$50,000")

In [ ]:
print(portfolio)

#### @chatprompt decorator

In [17]:
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage
from pydantic import BaseModel

class Quote(BaseModel):
    quote: str
    person: str


@chatprompt(
    SystemMessage("You are an avid reader of financial literature."),
    UserMessage("What is your favorite quote from Warren Buffet?"),
    AssistantMessage(
        Quote(
            quote="Price is what you pay; value is what you get.",
            person="Warren Buffet",
        )
    ),
    UserMessage("What is your favorite quote from {person}?"),
)
def get_finance_quote(person: str) -> Quote: ...


get_finance_quote("Charlie Munger")

Quote(quote='The big money is not in the buying and selling...but in the waiting.', person='Charlie Munger')

#### Function calling

In [19]:
import os
import requests
from magentic import prompt, FunctionCall
import logging
from openbb import obb
from langchain.tools import Tool
import reticker


def get_aggressive_smallcaps() -> list :
    return obb.equity.discovery.aggressive_small_caps(sort='desc').to_llm()

def get_sectors_performance() -> list:
    return obb.equity.compare.groups(group='sector', metric='performance', provider='finviz').to_llm()

def get_undervalued_growth() -> list:
    return obb.equity.discovery.undervalued_growth(provider='yfinance').to_llm()


def get_ticker_from_query(query):
    extractor = reticker.TickerExtractor(deduplicate=True)
    tickers = extractor.extract(query)
    if len(tickers) > 1:
        return ','.join(tickers)
    return tickers[0] 


def get_stock_price(query: str) -> str:
    """Get the current stock price for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.price.quote(symbol=ticker)

        return data.to_llm()
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

def get_company_overview(query: str) -> str:
    """Get an overview of a company for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.profile(symbol=ticker)
        return data.to_llm()
    except Exception as e:
        return f"Error fetching company overview for {ticker}: {str(e)}"

def get_latest_news_for_company(query : str) -> str:
    """ Get latest news for a company """
    try:
        ticker = get_ticker_from_query(query)
        return f"I dont have any news for now"
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

@prompt(
    "Use the appropriate search function to answer: {question}",
    functions=[get_stock_price, get_aggressive_smallcaps, get_sectors_performance, get_undervalued_growth],
)
def perform_search(question: str) -> FunctionCall[str]: ...


output = perform_search("Give me a list of tickers for aggressive smallcaps?")
output()

'[{"symbol":"SHCO","name":"Soho House & Co Inc.","price":6.49,"change":0.55,"percent_change":0.0925925,"volume":725882,"open":5.88,"high":6.57,"low":5.88,"previous_close":5.94,"ma50":7.3318,"ma200":5.90275,"year_high":8.47,"year_low":4.43,"market_cap":1259455744.0,"shares_outstanding":52560500.0,"book_value":-1.44,"price_to_book":-4.506944,"eps_ttm":-0.61,"eps_forward":-0.14,"pe_forward":-46.35714,"dividend_yield":0.0,"exchange":"NYQ","exchange_timezone":"America\\/New_York","earnings_date":"2024-12-19T13:24:00Z","currency":"USD"},{"symbol":"PTLO","name":"Portillo\'s Inc.","price":13.32,"change":1.0,"percent_change":0.08116883,"volume":1770774,"open":12.78,"high":13.375,"low":12.7101,"previous_close":12.32,"ma50":13.1498,"ma200":11.6184,"year_high":15.78,"year_low":8.38,"market_cap":850744384.0,"shares_outstanding":63869700.0,"book_value":6.299,"price_to_book":2.1146214,"eps_ttm":0.46,"eps_forward":0.39,"pe_forward":34.153847,"dividend_yield":0.0,"exchange":"NMS","exchange_timezone":"A

In [16]:
data = obb.equity.price.quote(symbol='XOM')
data.to_llm()

'[{"symbol":"XOM","name":"Exxon Mobil Corporation","exchange":"NYSE","last_price":113.76,"last_timestamp":"2025-03-17T20:00:52Z","open":111.8,"high":114.495,"low":111.75,"volume":19893152,"prev_close":111.9,"change":1.86,"change_percent":0.016622,"year_high":126.34,"year_low":103.67,"price_avg50":109.3972,"price_avg200":114.13215,"avg_volume":15834901,"market_cap":493620566400.0,"shares_outstanding":4339140000,"eps":7.84,"pe":14.51,"earnings_announcement":"2025-04-03T21:00:00Z"}]'

In [10]:
output()

'{"id":"067d91e3-7273-740d-8000-c29f742410fb","results":[{"symbol":"AAPL","asset_type":null,"name":"Apple Inc.","exchange":"NASDAQ","bid":null,"bid_size":null,"bid_exchange":null,"ask":null,"ask_size":null,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":214.0,"last_tick":null,"last_size":null,"last_timestamp":"2025-03-17T20:00:01Z","open":213.36,"high":215.22,"low":209.98,"close":null,"volume":47841607,"exchange_volume":null,"prev_close":213.49,"change":0.51,"change_percent":0.0023889,"year_high":260.1,"year_low":164.08,"price_avg50":234.2014,"price_avg200":227.89275,"avg_volume":53441279,"market_cap":3214729400000.0,"shares_outstanding":15022100000,"eps":6.97,"pe":30.7,"earnings_announcement":"2025-04-30T10:59:00Z"}],"provider":"fmp","warnings":null,"chart":null,"extra":{"metadata":{"arguments":{"provider_choices":{

In [11]:
output = perform_search('Please give me a list of latest aggressive smallcaps')
output()

OBBject

id: 067d91e5-6f58-7e9d-8000-dd95509a108a
results: [{'symbol': 'SHCO', 'name': 'Soho House & Co Inc.', 'price': 6.49, 'change...
provider: yfinance
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'yfinance'}, 's...

In [ ]:
output().to_df()

#### Prompt Chains

In [ ]:
import csv
from magentic import prompt_chain


def get_earnings_calendar(ticker: str, api_key: str = FMP_KEY) -> list:
    url = f"https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={ticker}&horizon=12month&apikey={api_key}"
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode("utf-8")
        cr = csv.reader(decoded_content.splitlines(), delimiter=",")
        my_list = list(cr)
    return my_list


@prompt_chain(
    "What's {ticker} expected earnings dates for the next 12 months?",
    functions=[get_earnings_calendar],
)
def get_earnings(ticker: str) -> str: ...


get_earnings("IBM")

#### Streaming Response

In [ ]:
from magentic import StreamedStr


@prompt("Explain to me {term} in a way a 5-year-old would understand.")
def describe_finance_term(term: str) -> StreamedStr: ...


# Print the chunks while they are being received
for chunk in describe_finance_term("liquidity"):
    print(chunk, end="")

#### Streaming Structured Outputs

In [ ]:
from collections.abc import Iterable
from time import time


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create {n_portfolio} portfolios with varying deegress of risk apetite.")
def create_portfolios(n_portfolio: int) -> Iterable[Portfolio]: ...


start_time = time()
for portfolio in create_portfolios(3):
    print(f"{time() - start_time:.2f}s : {portfolio}")

#### Asynchronous Streaming

In [ ]:
import asyncio
from typing import AsyncIterable


@prompt("List three high-growth stocks.")
async def iter_growth_stocks() -> AsyncIterable[str]: ...


@prompt("Tell me more about {stock_symbol}")
async def tell_me_more_about(stock_symbol: str) -> str: ...


start_time = time()
tasks = []
async for stock in await iter_growth_stocks():
    # Use asyncio.create_task to schedule the coroutine for execution before awaiting it
    # This way descriptions will start being generated while the list of stocks is still being generated
    task = asyncio.create_task(tell_me_more_about(stock))
    tasks.append(task)

descriptions = await asyncio.gather(*tasks)

for desc in descriptions:
    print(desc)